<a href="https://colab.research.google.com/github/lmassaron/fine-tuning-workshop/blob/main/sherlock_knowledge_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Check the GPU information
!nvidia-smi

Wed Sep 24 10:25:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.163.01             Driver Version: 550.163.01     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090        Off |   00000000:01:00.0  On |                  N/A |
|  0%   34C    P8             12W /  350W |    3663MiB /  24576MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Install necessary libraries for model training and evaluation
#%%capture
#!pip install -U transformers trl accelerate bitsandbytes

In [3]:
# Import and print the versions of the installed libraries
import torch
import trl
import bitsandbytes

print(f"Using PyTorch version: {torch.__version__}")
print(f"Using TRL version: {trl.__version__}")
print(f"Using bitsandbytes version: {bitsandbytes.__version__}")

Using PyTorch version: 2.8.0+cu128
Using TRL version: 0.22.2
Using bitsandbytes version: 0.47.0


In [4]:
# Import various libraries needed for data handling, model loading, and training
import os
import gc
import warnings
import torch
import numpy as np
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import login
from peft import LoraConfig
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import GRPOConfig, GRPOTrainer, SFTConfig, SFTTrainer

INFO 09-24 10:25:03 [__init__.py:216] Automatically detected platform cuda.


In [5]:
# Define configuration parameters for the model and data
class Config:
    """Configuration parameters"""

    SIZE = "3-1b"
    # MODEL_NAME = f"google/gemma-{SIZE}-it"
    MODEL_NAME = "lmassaron/gemma-3-1b-sherlock-expert"

    max_prompt_length = 352
    max_completion_length = 100

In [6]:
# Initialization script to set up the environment and Hugging Face login
def init():
    """Initialization script"""
    os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "0"
    os.environ["TOKENIZERS_PARALLELISM"] = "false"
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

    # It is recommended to set the HF_TOKEN as an environment variable
    token = os.environ.get("HF_TOKEN")
    if token:
        login(token=token)
    else:
      try:
        from google.colab import userdata
        # Retrieve your Hugging Face token from Colab's secrets manager
        # The name 'HF_TOKEN' should match the name you used in the secrets tab
        hf_token = userdata.get('HF_TOKEN')

        # Check if the token was successfully retrieved
        if hf_token:
            # Log in to Hugging Face using the retrieved token
            # The `add_to_git_credential=True` argument is optional and useful if you plan to push models to the Hub
            login(token=hf_token, add_to_git_credential=True)
            print("Hugging Face login successful using Google Colab secrets!")
        else:
            print("Error: HF_TOKEN not found in Google Colab secrets or is empty.")
            print("Please ensure you have created a secret named 'HF_TOKEN' in the 'Secrets' tab (🔑) on the left sidebar.")
      except:
        print("HF_TOKEN not set. You might need to log in manually.")

    torch.cuda.empty_cache()
    gc.collect()
    warnings.filterwarnings("ignore")

def is_bfloat16_supported():
    """Checks if the current device supports bfloat16."""
    return torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8


def info_device():
    """Get device for PyTorch"""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    return device

In [7]:
# Initialize the environment, get parameters, device, and data type
init()
params = Config()
device = info_device()
dtype = torch.bfloat16 if is_bfloat16_supported() else torch.float16
print(f"Using dtype: {dtype}")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Using device: cuda
Using dtype: torch.bfloat16


In [8]:
# Function to load dataset from Hugging Face Hub
def get_data(repo_id, mapping_func=None, split="train"):
    """Upload HF dataset"""
    data = load_dataset(repo_id, cache_dir="/tmp")[split]
    if mapping_func:
      data = data.map(mapping_func)
    return data

In [9]:
# Load the Sherlock QA dataset
data = get_data(repo_id="lmassaron/Sherlock_QA_test")

In [10]:
# Display the loaded dataset information
data

Dataset({
    features: ['Question', 'Answer', 'Difficulty'],
    num_rows: 25
})

In [11]:
# Load the tokenizer and model from Hugging Face
tokenizer = AutoTokenizer.from_pretrained(params.MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    params.MODEL_NAME,
    torch_dtype=dtype,
    device_map=device,
    use_cache=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

In [12]:
# Evaluate the model on the dataset and store results
temperature = 0
results_list = []
instructions = "\nBriefly, just give the straight answer to the question."

# It's good practice to set the pad_token if it's not already set.
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

for row in tqdm(data, desc="Evaluating Samples"):
  question = row['Question']
  answer = row['Answer']
  difficulty = row['Difficulty']

  # Tokenize the input and get both input_ids and attention_mask
  inputs = tokenizer.apply_chat_template(
            [{"role": "user", "content": question + instructions}],
            tokenize=True,
            add_generation_prompt=True,  # Crucial for telling the model it's its turn to speak
            return_tensors="pt",
            return_dict=True  # Ensure the output is a dictionary
        ).to(device)

  # Prepare arguments for the generate function
  generation_kwargs = {
      "pad_token_id": tokenizer.eos_token_id,
      "max_new_tokens": params.max_completion_length,
      "do_sample": temperature > 0
  }

  # Only add temperature to kwargs if sampling is enabled
  if generation_kwargs["do_sample"]:
      generation_kwargs["temperature"] = temperature

  # Generate a completion from the model, passing the attention_mask
  outputs = model.generate(
      inputs.input_ids, # Pass input_ids explicitly
      attention_mask=inputs.attention_mask, # Pass the attention mask
      **generation_kwargs
      )

  generated_token_ids = outputs[0, inputs.input_ids.shape[-1] :]
  generated_text = tokenizer.decode(
      generated_token_ids,
      skip_special_tokens=True,
  ).strip()

  results_list.append({
      'question': question,
      'expected_answer': answer,
      'generated_answer': generated_text,
      'difficulty': difficulty
  })

results_df = pd.DataFrame(results_list)

Evaluating Samples:   8%|▊         | 2/25 [00:52<10:25, 27.18s/it]The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Evaluating Samples:  16%|█▌        | 4/25 [00:52<03:33, 10.17s/it]The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Evaluating Samples:  24%|██▍       | 6/25 [01:10<02:59,  9.46s/it]The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info

In [13]:
# Delete the model and tokenizer to free up GPU memory
del [model, tokenizer]

In [14]:
# Evaluate correctness based on keyword matching
def evaluate_keyword(row):
    return row['expected_answer'].lower() in row['generated_answer'].lower()

results_df['is_correct_keyword'] = results_df.apply(evaluate_keyword, axis=1)


In [17]:
# Evaluate correctness based on semantic similarity using Sentence-BERT
from sentence_transformers import SentenceTransformer, util

# Load the Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode the expected and generated answers into embeddings
expected_embeddings = model.encode(results_df['expected_answer'].tolist(), convert_to_tensor=True)
generated_embeddings = model.encode(results_df['generated_answer'].tolist(), convert_to_tensor=True)

# Calculate cosine similarity between embeddings
cosine_scores = util.cos_sim(expected_embeddings, generated_embeddings)
cosine_scores = np.array(cosine_scores.cpu())

# Store the semantic similarity scores
results_df['semantic_similarity'] = [cosine_scores[i][i] for i in range(len(cosine_scores))]

# Determine correctness based on a similarity threshold
similarity_threshold = 0.5
results_df['is_correct_semantic'] = results_df['semantic_similarity'] >= similarity_threshold

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:
# Delete the Sentence-BERT model to free up memory
del [model]

In [20]:
# Load the evaluation model and tokenizer (AI Judge)
evaluation_model = "meta-llama/Llama-3.2-3B-Instruct" # "alpindale/Llama-3.2-3B-Instruct"
eval_tokenizer = AutoTokenizer.from_pretrained(evaluation_model)
eval_model = AutoModelForCausalLM.from_pretrained(
    evaluation_model,
    torch_dtype=dtype,
    device_map=device,
    use_cache=True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [21]:
# Function to generate the prompt for the AI judge
def evaluation_prompt(question, expected_answer, generated_answer):
  prompt = f"""You are an impartial evaluator.
Your task is to determine if the "Generated Answer", even if too verbose, correctly answers the "Question".
The "Expected Answer" is provided as a reference for the correct information.

Question:
{question}

Expected Answer:
{expected_answer}

Generated Answer:
{generated_answer}

Is the "Generated Answer" correct? Please answer with "Yes" or "No".
"""
  return prompt

# Evaluate generated answers using the AI judge
ai_judge = []

for i in tqdm(range(len(results_df))):
  question = results_df.iloc[i]['question']
  expected_answer = results_df.iloc[i]['expected_answer']
  generated_answer = results_df.iloc[i]['generated_answer']
  prompt = evaluation_prompt(question, expected_answer, generated_answer)

  inputs = eval_tokenizer.apply_chat_template(
            [{"role": "user", "content": prompt}],
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
        ).to(device)

  # Generate a response from the AI judge
  outputs = eval_model.generate(
      inputs,
      pad_token_id=eval_tokenizer.eos_token_id,
      max_new_tokens=100,
      temperature=0.1,
      do_sample=True,
  )

  generated_token_ids = outputs[0, inputs.shape[-1] :]
  generated_text = eval_tokenizer.decode(
      generated_token_ids,
      skip_special_tokens=True,
  ).strip()

  # Determine correctness based on the AI judge's response
  if "yes" in generated_text.lower():
    ai_judge.append(True)
  else:
    ai_judge.append(False)

results_df["is_correct_ai_eval"] = ai_judge

100%|██████████| 25/25 [00:03<00:00,  8.18it/s]


In [22]:
# Calculate overall correctness metrics for each evaluation method
overall_keyword_accuracy = results_df['is_correct_keyword'].mean()
overall_semantic_accuracy = results_df['is_correct_semantic'].mean()
overall_ai_judge_accuracy = results_df['is_correct_ai_eval'].mean()

print(f"Overall Keyword Matching Accuracy: {overall_keyword_accuracy:.2f}")
print(f"Overall Semantic Similarity Accuracy (threshold=0.5): {overall_semantic_accuracy:.2f}")
print(f"Overall AI Judge Accuracy: {overall_ai_judge_accuracy:.2f}")

Overall Keyword Matching Accuracy: 0.24
Overall Semantic Similarity Accuracy (threshold=0.5): 0.44
Overall AI Judge Accuracy: 0.36


In [23]:
# Analyze correctness by difficulty for each evaluation method
difficulty_analysis_keyword = results_df.groupby('difficulty')['is_correct_keyword'].mean().reset_index()
difficulty_analysis_semantic = results_df.groupby('difficulty')['is_correct_semantic'].mean().reset_index()
difficulty_analysis_ai_judge = results_df.groupby('difficulty')['is_correct_ai_eval'].mean().reset_index()

print("\nKeyword Matching Accuracy by Difficulty:")
display(difficulty_analysis_keyword)

print("\nSemantic Similarity Accuracy by Difficulty (threshold=0.5):")
display(difficulty_analysis_semantic)

print("\nAI Judge Accuracy by Difficulty:")
display(difficulty_analysis_ai_judge)


Keyword Matching Accuracy by Difficulty:


,difficulty,is_correct_keyword
0,Easy,0.250000
1,Hard,0.000000
2,Medium,0.333333



Semantic Similarity Accuracy by Difficulty (threshold=0.5):


,difficulty,is_correct_semantic
0,Easy,0.500000
1,Hard,0.250000
2,Medium,0.444444



AI Judge Accuracy by Difficulty:


,difficulty,is_correct_ai_eval
0,Easy,0.500000
1,Hard,0.000000
2,Medium,0.333333


In [24]:
# Display the detailed results DataFrame
display(results_df)

,question,expected_answer,generated_answer,difficulty,is_correct_keyword,semantic_similarity,is_correct_semantic,is_correct_ai_eval
0,Who created the character of Sherlock Holmes?,Sir Arthur Conan Doyle,Sir Arthur Conan Doyle,Easy,True,1.000000,True,True
1,What is the name of Sherlock Holmes's enemy?,Professor Moriarty,Jack the Ripper,Easy,False,0.283280,False,False
2,Where does Sherlock Holmes live?,221b Baker Street in London,221B Baker Street,Easy,False,0.912416,True,True
3,Who is Sherlock Holmes's best friend?,Dr. John Watson,Mycroft Holmes,Easy,False,0.492548,False,False
4,What is the name of Sherlock's older brother?,Mycroft Holmes,Mycroft,Easy,False,0.842383,True,True
5,Who is the landlady of 221b Baker Street?,Mrs. Hudson,Mrs. Hudson,Easy,True,1.000000,True,True
6,What musical instrument does Sherlock Holmes l...,The violin,A violin,Easy,False,0.939922,True,True
7,In which Sherlock Holmes short story do we mee...,A Scandal In Bohemia,A Scandal in Bohemia,Medium,True,1.000000,True,True
8,Which actor plays Sherlock Holmes in the TV se...,Benedict Cumberbatch,Benedict Cumberbatch,Easy,True,1.000000,True,True
9,Who did Dr. Watson marry?,Mary Morstan,Mary Watson,Medium,False,0.618045,True,False


In [ ]:
from tenacity import retry, stop_after_attempt, wait_fixed

# Function to load dataset from Hugging Face Hub with retries
@retry(stop=stop_after_attempt(5), wait=wait_fixed(10)) # Retry up to 5 times with 10 seconds delay
def get_data(repo_id, mapping_func=None, split="train"):
    """Upload HF dataset with retries"""
    print(f"Attempting to load dataset {repo_id}, split {split}...")
    data = load_dataset(repo_id, cache_dir="/tmp")[split]
    if mapping_func:
      data = data.map(mapping_func)
    print("Dataset loaded successfully!")
    return data

In [ ]:
data = get_data(repo_id="lmassaron/Sherlock_QA")
test_data = get_data(repo_id="lmassaron/Sherlock_QA_test")

for k, item in enumerate(test_data):
    answer = item["Answer"].lower()
    counter = 0
    for messages in data:
        for content in messages["messages"]:
            if content["role"] == "assistant":
                if answer in content["content"].lower():
                    counter += 1
    print(
        f"The answer to the question {k} ({item["Question"][:50]}) appears {counter} times in the training data"
    )

question                     What is Sherlock Holmes's most famous line?
expected_answer                            "Elementary, my dear Watson."
generated_answer       'The man who is born to be a detective is as r...
difficulty                                                          Easy
is_correct_keyword                                                 False
semantic_similarity                                             0.280917
is_correct_semantic                                                False
is_correct_ai_eval                                                 False
Name: 22, dtype: object